In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os

In [2]:
dataset = fo.Dataset.from_images_dir("F:\\AI_Challenge\\keyframes", name="AI_Challenge", tags=None, recursive=True)
#tạo dataset mới

In [3]:
#dataset = fo.load_dataset("test")

In [2]:
dataset = fo.load_dataset("AI_Challenge")
#load dataset

In [4]:
session = fo.launch_app(dataset, auto=False)
session.open_tab()
#khởi chạy fiftyone từ dữ liệu dataset

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [6]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    sample.save()
#gán video, frameid vào sample

In [7]:
#object dêtction
for sample in dataset:
    object_path = f"E:\\AI_Challenge\\objects\\{sample['video']}\\{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()

In [15]:
all_keyframe = glob('F:\\AI_Challenge\\keyframes\\*\\*.jpg')
video_keyframe_dict = {}
all_video = glob('F:\\AI_Challenge\\keyframes\\*')
all_video = [v.rsplit('\\',1)[-1] for v in all_video]

In [16]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

In [17]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

In [18]:
embedding_dict = {}
for v in all_video:
    clip_path = f'E:\\AI_Challenge\\clip-features-vit-b32\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]

In [19]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)

In [20]:
#CLIP feature
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

In [2]:
print(fo.list_datasets())
#in danh sách dataset

['AI_Challenge', 'test']


In [ ]:
fo.core.session.close_app()
#đóng fiftyone

In [21]:
import csv

In [22]:
#thêm frame idx và time
all_video = glob("F:\\AI_Challenge\\keyframes\\*")
for i in all_video:
    #file video
    video_id=i[-8:]
    all_keyframe = glob("F:\\AI_Challenge\\keyframes\\"+video_id+"\\*.jpg")
    
    table=[]
    with open('E:\\AI_Challenge\\map-keyframes\\'+ video_id +'.csv') as file:
        csvreader = csv.reader(file)
     
    # extracting field names through first row
        fields = next(csvreader)
 
    # extracting each data row one by one
        for row in csvreader:
            table.append(row)

    for frame in all_keyframe:
        #file frame
        sample = dataset[frame]
        row = int(sample['frameid'])-1
        sample['frame_idx'] = table[row][3]
        time = float(table[row][1])
        sample['time'] = (str(int(time//60))+ ':' + str(int(time%60)))
        sample.save()
